<a href="https://colab.research.google.com/github/spencerklawans/466-Project/blob/main/WildfirePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wildfire Prediction
Brought to you by Gooogle Earth Engine

In [1]:
# Init Google Earth Engine
import ee
ee.Authenticate()
ee.Initialize()

ModuleNotFoundError: No module named 'ee'

In [1]:
#Init KDD libraries

%matplotlib inline

import numpy as np
import pandas as pd
import sklearn

RuntimeError: Polyfit sanity test emitted a warning, most likely due to using a buggy Accelerate backend. If you compiled yourself, see site.cfg.example for information. Otherwise report this to the vendor that provided NumPy.
RankWarning: Polyfit may be poorly conditioned
